# Problem 74

The number 145 is well known for the property that the sum of the
factorial of its digits is equal to 145:

    1! + 4! + 5! = 1 + 24 + 120 = 145

Perhaps less well known is 169, in that it produces the longest chain of
numbers that link back to 169; it turns out that there are only three such
loops that exist:

    169 → 363601 → 1454 → 169
    871 → 45361 → 871
    872 → 45362 → 872

It is not difficult to prove that EVERY starting number will eventually
get stuck in a loop. For example,

    69 → 363600 → 1454 → 169 → 363601 (→ 1454)
    78 → 45360 → 871 → 45361 (→ 871)
    540 → 145 (→ 145)

Starting with 69 produces a chain of five non-repeating terms, but the
longest non-repeating chain with a starting number below one million is
sixty terms.

How many chains, with a starting number below one million, contain exactly
sixty non-repeating terms?

In [1]:
from math import factorial

factorial_cache = [factorial(digit) for digit in xrange(10)]

In [2]:
def sum_of_factorial_of_digits(number):
    return sum(factorial_cache[int(digit)] for digit in str(number))

In [3]:
def compute_chain(number):
    
    chain = [number]
    
    number = sum_of_factorial_of_digits(number)
    
    while number not in chain:
        chain.append(number)
        number = sum_of_factorial_of_digits(number)
        
    return chain

In [4]:
from tqdm import trange

sum(1 for n in trange(1000000) if len(compute_chain(n)) == 60)

100%|██████████| 1000000/1000000 [02:22<00:00, 7035.53it/s]


402

This is a faster solution. When we compute a chain we cache the resulting chain. By doing that, we avoid computing partial chains when we find a number that is already in cache. We approx get a speedup of > 4x.

In [47]:
cache = {}

def add_while_not_present(chain, cached_values):
    for c in cached_values:
        if c not in chain:
            chain.append(c)
        else:
            break

def compute_chain_cached(number):
    
    if number in cache:
        return cache[number]
    
    chain = [number]
    
    number = sum_of_factorial_of_digits(number)
    if number in cache:
        add_while_not_present(chain, cache[number])
        return chain
    
    while number not in chain:
        if number in cache:
            add_while_not_present(chain, cache[number])
            break
            
        chain.append(number)
        number = sum_of_factorial_of_digits(number)
    
    if number not in cache:
        cache[number] = chain[chain.index(number):]
        
    cache[chain[0]] = chain
        
    return chain

In [48]:
from tqdm import trange

sum(1 for n in trange(1000000) if len(compute_chain_cached(n)) == 60)

100%|██████████| 1000000/1000000 [00:29<00:00, 33654.04it/s]


402